In [1]:
import pandas as pd
%matplotlib inline
import sklearn
from sklearn import *
from sklearn.preprocessing import LabelEncoder,LabelBinarizer,OneHotEncoder
import numpy as np
import os
from scipy import sparse
import scipy
    
import scipy.sparse as sp

In [2]:
#read five tables
df_coupon_train=pd.read_csv('input/coupon_list_train.csv')
df_coupon_test=pd.read_csv('input/coupon_list_test.csv')
df_coupon_buy=pd.read_csv('input/coupon_detail_train.csv')
df_coupon_visit=pd.read_csv('input/coupon_visit_train.csv')
df_user=pd.read_csv('input/user_list.csv')

In [ ]:
df_coupon_train.columns

In [ ]:
df_coupon_visit.T

In [3]:
script_path = os.path.abspath(os.path.dirname('__file__'))

import pandas as pd
import numpy as np
import os
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.preprocessing import StandardScaler

def extract_topn_item4user(df_itemUser_groupby_user, n=10, column="predict", merge_column="COUPON_ID_hash"):
    '''
    get top n row
    :param pandas.DataFrame df:
    :param int n:
    :param str column:
    :rtype: pandas.DataFrame
    '''
    return " ".join(df_itemUser_groupby_user.sort_index(by=column)[-n:][merge_column])

def binarizeFrame(df):
    dic=df.T.to_dict().values()
    vectorizer = DV( sparse = False )
    vec_x_cat_train = vectorizer.fit_transform( dic)
    return vec_x_cat_train

class Get_Match_Pref(BaseEstimator, TransformerMixin):
    '''
    get user pref is match coupon area
    '''

    def get_feature_names(self):

        return [self.__class__.__name__]

    def fit(self, date_frame, y=None):
        '''
        fit

        :param pandas.DataFrame: all data
        :rtype: Get_Price_Rate
        '''

        return self

    def transform(self, date_frame):
        '''
        transform

        :param pandas.DataFrame: all data
        :rtype: array
        '''
        res_sr = date_frame["PREF_NAME"] == date_frame["ken_name"]

        return res_sr.as_matrix()[None].T.astype(np.float)

class Get_user_feature(BaseEstimator, TransformerMixin):
    
    def get_feature_names(self):

        return [self.__class__.__name__]

    def fit(self, data_frame, y=None):
        '''
        fit
        :param pandas.DataFrame: all data
        :rtype: Get_Price_Rate
        '''
        return self
    
    def transform(self, df_coupon_train):
        '''
        transform
        :param pandas.DataFrame: all data
        :rtype: array
        '''
        le=LabelEncoder()
        df_coupon_train['SEX_ID_Num']=le.fit_transform(df_coupon_train['SEX_ID'])
        return df_coupon_train[['SEX_ID_Num', 'AGE','user_global_index']].as_matrix().astype(np.float)


          
class Get_coupon_feature(BaseEstimator, TransformerMixin):
    def encode(self,v):
        u = v.unique()
        lm = len(u) + 1
        d = {np.nan:lm, "nan": lm}  
        for ii, iu in enumerate(u):
            d[iu] = ii
        return v.apply(lambda x: d.get(x, lm) ).values

    def binarizeCategorical(self,df_coupon_cate):
        oh=OneHotEncoder()
        res_final = None
        for i, col in enumerate(df_coupon_cate.columns):
            #print col
            res = self.encode(df_coupon_cate[col])
            #print res.shape, np.unique(res), 
            #print len(df_coupon_train_cat[col].unique())
            res = oh.fit_transform(res.reshape(len(res), 1))
            res_final = res if res_final is None else sp.hstack((res_final, res))
            #print res_final.shape
        return res_final.todense()
    
    
    def get_feature_names(self):

        return [self.__class__.__name__]

    def fit(self, df_coupon_train_cat, y=None):
        '''
        fit
        :param pandas.DataFrame: all data
        :rtype: Get_Price_Rate
        '''
        #df_coupon_train_cat.apply(lambda x: len(x.unique()))
        return self
    
    def transform(self, df_coupon):
        '''
        transform
        :param pandas.DataFrame: coupon property pandas data frame
        :rtype: array
        '''
        #'COUPON_ID_hash',
        df_coupon_nume=df_coupon[['PRICE_RATE', 'CATALOG_PRICE','DISCOUNT_PRICE','VALIDPERIOD','DISPPERIOD',\
                                              'USABLE_DATE_MON', 'USABLE_DATE_TUE',\
               'USABLE_DATE_WED', 'USABLE_DATE_THU', 'USABLE_DATE_FRI',\
               'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY','USABLE_DATE_BEFORE_HOLIDAY']]
        df_coupon_cate=df_coupon[['CAPSULE_TEXT', 'GENRE_NAME', 'large_area_name', 'ken_name','small_area_name']]
        
        #sparse_m=binarizeCategorical(df_coupon_train_cat)
        #sp.hstack(sparse_m,df_coupon_train_num.values)
        df_coupon_binary=pd.DataFrame(self.binarizeCategorical(df_coupon_cate))  
        df_coupon_combine=df_coupon_nume.join(df_coupon_binary)
        #df_coupon_combine.set_index('COUPON_ID_hash',inplace=True)
        df_coupon_combine=df_coupon_combine.fillna(0)
        #ss=StandardScaler()
        #df_coupon_train_combine.apply(lambda x:ss.fit_transform(x))
        return df_coupon_combine.as_matrix().astype(np.float)

In [ ]:
np.nonzero(pd.DataFrame(Xs)[162].values)

In [4]:
# setting rating matrix to be the number of buying
df_rating=df_coupon_buy[[u'USER_ID_hash',u'COUPON_ID_hash','ITEM_COUNT']]
# transform the hash code to natural index in df_coupon_train
df_user['user_global_index']=df_user.index # all the users now has a unique index
df_coupon_train['coupon_global_index']=df_coupon_train.index # all the coupons in training set now has a unique index
#df_CouponProperty=pd.concat([df_coupon_train,df_coupon_test])

# build up user global index and coupon global index as row and column index for the sparse-matrix
user_id=df_user.set_index('USER_ID_hash').loc[df_rating['USER_ID_hash']]['user_global_index'].values
coupon_id=df_coupon_train.set_index('COUPON_ID_hash').loc[df_rating['COUPON_ID_hash']]['coupon_global_index'].values
num_item,num_user=len(df_coupon_train.COUPON_ID_hash.unique()),len(df_user.USER_ID_hash.unique())
V=df_rating.ITEM_COUNT
Xu=sparse.coo_matrix((V,(coupon_id,user_id)),shape=(num_item,num_user)) # the user_id looks like (1,1,1,2,2,3,4,4,...)


In [ ]:
#negative row indexs
# transform the hash code to natural index in df_coupon_train
df_coupon_visit.drop(labels=[u'I_DATE',u'PAGE_SERIAL',u'REFERRER_hash',u'SESSION_ID_hash',u'PURCHASEID_hash'],axis=1,inplace=True)
#g_visit=df_coupon_visit.groupby('VIEW_COUPON_ID_hash')
#g_visit.count()
g_visit2=df_coupon_visit.groupby(['VIEW_COUPON_ID_hash','USER_ID_hash'])
#df_rating=pd.DataFrame(g_visit2.apply(lambda x: np.sum(x['PURCHASE_FLG'])))
df_rating=df_coupon_visit[[u'USER_ID_hash',u'VIEW_COUPON_ID_hash','PURCHASE_FLG']] #duplicate coupon??????
df_user['user_global_index']=df_user.index # all the users now has a unique index
df_coupon_train['coupon_global_index']=df_coupon_train.index # all the coupons in training set now has a unique index
#df_CouponProperty=pd.concat([df_coupon_train,df_coupon_test])
# build up user global index and coupon global index for row and column index for sparse matrix
user_id=df_user.set_index('USER_ID_hash').loc[df_rating['USER_ID_hash']]['user_global_index'].values
coupon_id=df_coupon_train.set_index('COUPON_ID_hash').loc[df_rating['VIEW_COUPON_ID_hash']]['coupon_global_index'].values
num_item,num_user=len(df_coupon_train.COUPON_ID_hash.unique()),len(df_user.USER_ID_hash.unique())
V=df_rating.PURCHASE_FLG
Xu=sparse.coo_matrix((V,(coupon_id,user_id)),shape=(num_item,num_user)) # the user_id looks like (1,1,1,2,2,3,4,4,...)

In [ ]:
df_rating=pd.DataFrame(g_visit2.agg(lambda x: np.sum(x['PURCHASE_FLG'])))

In [5]:
feature_list = [('item_feature', Get_coupon_feature())]
fu_obj = FeatureUnion(transformer_list=feature_list)
Xs=fu_obj.fit_transform(pd.concat([df_coupon_train,df_coupon_test],axis=0)) # df_coupon_train is already the natural global index for items
Xs_train=Xs[0:19413,:]
Xs_test=Xs[19413:,:]

In [ ]:
np.nonzero(Xs[0:19413,-1]),np.nonzero(Xs[19413:19413+310,-1])

In [ ]:
df_coupon_train.shape,df_coupon_test.shape,Xs.shape,Xs_test.shape

In [ ]:
#the following is not good way to transform hash code to index
'''
userIndex=label_encoder.fit_transform(df_user.USER_ID_hash.unique())
J=label_encoder.transform(df_rating.USER_ID_hash)
itemIndex=label_encoder.fit_transform(df_coupon_train.COUPON_ID_hash.unique())
I=label_encoder.transform(df_rating.COUPON_ID_hash)
V=df_rating.ITEM_COUNT
Xu= sparse.coo_matrix((V,(I,J)),shape=(len(itemIndex),len(userIndex)))
'''
Xs.shape

In [8]:
import pickle
with open('Xstr_Xu_Xs_te.pickle', 'wb') as f:
            pickle.dump((Xs_train,Xu,Xs_test), f)


In [9]:
with open('Xstr_Xu_Xs_te.pickle', 'rb') as f:
    Xs_train,Xu,Xs_test=pickle.load(f)

In [10]:
Xs_train.shape,Xu.shape,Xs_test.shape

((19413, 163), (19413, 22873), (310, 163))

In [ ]:
import lce
reload(lce)
OBJ=lce.LCE()
OBJ.run(sparse.coo_matrix(Xs),Xu)

In [ ]:
#joint three dataframes: df_coupon_visit*df_coupon_train(coupon information)*df_user(user information)
train_df = pd.merge(df_coupon_visit, df_coupon_train,left_on="VIEW_COUPON_ID_hash", right_on="COUPON_ID_hash")
train_df = pd.merge(train_df, df_user,left_on="USER_ID_hash", right_on="USER_ID_hash")

In [ ]:
feature_list = [('item_feature', Get_coupon_feature()),('user_item',Get_Match_Pref()),('getUserFeature',Get_user_feature())]
fu_obj = FeatureUnion(transformer_list=feature_list)
X_train = fu_obj.fit_transform(train_df)

In [ ]:
y_train = train_df["PURCHASE_FLG"]
assert X_train.shape[0] == y_train.size
# fit model
from sklearn import ensemble
original_params = {'n_estimators': 1000,'n_jobs':-1}
params = dict(original_params)
clf=ensemble.RandomForestClassifier(**params)
#clf = ensemble.GradientBoostingClassifier(**params)
clf.fit(X_train, y_train)
# create test_df

In [ ]:
#generate the Cartesian Product
df_coupon_test["cross"] = 1 # all test items are new, ie, cold-start items
df_user["cross"] = 1 # all users are old
test_df_item_user = pd.merge(df_coupon_test,df_user, on="cross")
# create test Feature
X_test = fu_obj.transform(test_df_item_user)
# predict test data


In [ ]:
predict_proba = clf.predict_proba(X_test)
pos_idx = np.where(clf.classes_ == True)[0][0] # find out which column is positive
test_df_item_user["predict"] = predict_proba[:, pos_idx] 

In [ ]:
top10_coupon=test_df_item_user.groupby("USER_ID_hash").apply(extract_topn_item4user) #for each user
top10_coupon.name = "PURCHASED_COUPONS"
top10_coupon.to_csv("submission.csv", header=True)

In [ ]:
'''
X_train.shape,y_train.shape
pd.unique(df_coupon_train.USABLE_DATE_FRI)
df_rating['user_global_index'] = pd.Series(user_id, index=df_rating.index)
df_rating['coupon_global_index']=pd.Series(coupon_id, index=df_rating.index)
df_rating=df_rating.drop(labels=['USER_ID_hash','COUPON_ID_hash'],axis=1)
df_rating2['user_global_index'] = pd.Series(user_id, index=df_rating.index)
df_rating['coupon_global_index']=pd.Series(coupon_id, index=df_rating.index)
df_rating=df_rating.drop(labels=['USER_ID_hash','COUPON_ID_hash'],axis=1)
pd.Series(coupon_id).hist()
df_rating2[['USER_ID_hash','VIEW_COUPON_ID_hash','PURCHASE_FLG']].describe()
df_CouponProperty=pd.read_csv('coupon_list_train.csv')

trainCouponSet=set(df_coupon_train.COUPON_ID_hash.unique()) import lce eng=lce.LCE() testCouponSet=set(df_coupon_test.COUPON_ID_hash.unique()) len(trainCouponSet) len(testCouponSet) df_coupon_buy.ITEM_COUNT.unique() trainCouponSet.intersection(testCouponSet) len(set(df_coupon_visit.VIEW_COUPON_ID_hash.unique()).intersection(set(df_coupon_buy.COUPON_ID_hash.unique()))) trainCouponSet.intersection(set(df_coupon_visit.VIEW_COUPON_ID_hash.unique()))

'''



In [ ]:
from scipy import sparse
from scipy.sparse import spdiags
data = np.array([[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]])
diags = np.array([0, -1, 2])
data*data
np.multiply(data,data)
data,sp*data
1. / (np.sqrt(np.sum(np.multiply(data,data), axis=1)))
sp=sparse.spdiags(1./np.sqrt(np.sum(data * data, axis=1)),0,data.shape[0],data.shape[0])
sp.todense()*data
np.sum(data,axis=1)
####################################
sparse.coo_matrix.multiply(Xu,Xu.toarray())

Xu.sum(1).shape
(1. / (np.sqrt(Xu.multiply(Xu).sum(1)) )).shape
1. / np.sqrt((Xu.multiply(Xu).sum(1)) ).T
sparse.spdiags((1. / np.sqrt((Xu.multiply(Xu).sum(1)) )).T, 0, Xu.shape[0], Xu.shape[0]).todense()
sparse.spdiags?

In [ ]:
row  = np.array([0, 3, 1, 0])
col  = np.array([0, 3, 1, 2])
data = np.array([4, 5, 7, 9])
A=sparse.coo_matrix((data, (row, col)), shape=(4, 4))


In [ ]:
row = np.array([0, 2, 2, 0 ])
col = np.array([0, 0, 1, 2])
data = np.array([1.0, 2.0, 3.0, 4.0])
B=sparse.csc_matrix((data, (row, col)), shape=(4, 4))

In [ ]:
AA=sparse.csc_matrix(A)

In [ ]:
BB=sparse.csc_matrix(B)

In [ ]:
sparse.csc_matrix.multiply(AA,BB.T).todense()

In [ ]:
BB.todense(),AA.todense()

In [ ]:
BB.shape

In [ ]:
(BB+BB).todense(),BB.toarray()

In [ ]:
scipy.__version__

In [ ]:
reload(scipy)

In [ ]:
np.maximum(BB.toarray(),10)

In [ ]:
sparse.csc_matrix.maximum(BB,1).toarray()

In [ ]:
 print("Hu is %s" %BB)

In [ ]:
BB.diagonal()

In [ ]:
np.sum(BB.diagonal())

In [ ]:
df_coupon_test

In [ ]:
Xs_test.shape

In [ ]:
import lce

In [ ]:
import pickle
with open('Xs_test.pickle', 'wb') as f:
    pickle.dump(Xs_test, f)